### Etapas:

   **Passo 1** - Instalar e importar bibliotecas.
   
   **Passo 2** - Extrair dados do ETF.com utilizando um navegador.
   
   **Passo 3** - Extrair dados necessários através de algum identificador (caminho do htlm, por exemplo), armazenando, ao fim, todos os dados coletados.
   
   **Passo 4** - Fazer tratamento inicial dos dados, gerando um output final com uma base de dados a ser utilizada (ETAPA MAIS IMPORTANTE).

### **Passo 1** - Instalar e importar bibliotecas.

In [1]:
!pip install webdriver-manager
!pip install selenium
!pip install html5lib


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

### **Passo 2** - Extrair dados do ETF.com utilizando um navegador.

In [2]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

url = "https://www.etf.com/etfanalytics/etf-screener"

driver.get(url)

### **Passo 3** - Extrair dados necessários através de algum identificador (caminho do htlm, por exemplo), armazenando, ao fim, todos os dados coletados.

In [3]:
# Expandindo visualização da tabela para 100 itens

# time.sleep(15)

botao_100 = driver.find_element("xpath", "/html/body/div[2]/div/div[1]/main/div/section/div[2]/div[2]/div[3]/div/article/div/div[3]/div/div[1]/div/div/div/div[3]/div[2]/div/div[1]/div/div[2]/div[1]/div/div[5]/button")

driver.execute_script("arguments[0].click();", botao_100)

In [4]:
# Pegando número total de tabelas que o site possuir

numero_paginas = driver.find_element("xpath", "/html/body/div[2]/div/div[1]/main/div/section/div[2]/div[2]/div[3]/div/article/div/div[3]/div/div[1]/div/div/div/div[3]/div[2]/div/div[1]/div/div[2]/div[2]/ul/li[8]/a")
numero_paginas = numero_paginas.text.strip()
numero_paginas = int(numero_paginas)

print(numero_paginas)

34


In [5]:
# Realizando primeira extração de todos os dados

lista_dados = []

for pagina in range(1, numero_paginas+1):

    tabela = driver.find_element("xpath", "/html/body/div[2]/div/div[1]/main/div/section/div[2]/div[2]/div[3]/div/article/div/div[3]/div/div[1]/div/div/div/div[3]/div[2]/div/div[1]/div/div[1]/table")
    html_tabela = tabela.get_attribute("outerHTML")
    tabela_final = pd.read_html(html_tabela)[0]
    
    lista_dados.append(tabela_final)
    
    if pagina < numero_paginas:
        avancar = driver.find_element("xpath", "//a[@class='page-link' and text()='Next']")
        driver.execute_script("arguments[0].click();", avancar)

In [6]:
# Mudando para a aba de performance

botao_performance = driver.find_element("xpath", "/html/body/div[2]/div/div[1]/main/div/section/div[2]/div[2]/div[3]/div/article/div/div[3]/div/div[1]/div/div/div/div[3]/div[2]/div/ul/li[2]")

driver.execute_script("arguments[0].click();", botao_performance)

In [7]:
# Lendo todas as tabelas novamente

lista_performance = []

for pagina in range(1, numero_paginas+1):

    tabela = driver.find_element("xpath", "//table[@id='screener-output-table']")
    html_tabela = tabela.get_attribute("outerHTML")
    tabela_final = pd.read_html(html_tabela)[0]

    lista_performance.append(tabela_final)
    
    if pagina < numero_paginas:
        avancar = driver.find_element("xpath", "//a[@class='page-link' and text()='Next']")
        driver.execute_script("arguments[0].click();", avancar)

In [8]:
#fechando navegador

driver.quit()

In [9]:
# Utilizando o Concat

base_dados = pd.concat(lista_dados, axis=0, join='outer', ignore_index=True)
display(base_dados)

base_performance = pd.concat(lista_performance, axis=0, join='outer', ignore_index=True)
display(base_performance)

,Ticker,Fund Name,Segment,1 YR Return,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,29.29%,0.09%,$491.91B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,29.46%,0.03%,$433.99B
2,VOO,Vanguard 500 Index Fund,Equity: U.S. - Large Cap,29.41%,0.03%,$404.83B
3,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,27.50%,0.03%,$366.51B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,49.90%,0.20%,$244.72B
...,...,...,...,...,...,...
3395,BINV,Brandes International ETF,Equity: Global - Total Market Value,--,0.70%,--
3396,BUSA,Brandes U.S. Value ETF,Equity: U.S. - Total Market Value,--,0.60%,--
3397,BSMC,Brandes US Small-Mid Cap Value ETF,Equity: U.S. - Extended Market Value,--,0.70%,--
3398,MSTY,YieldMax MSTR Option Income Strategy ETF,Equity: U.S. Application Software,--,--,--


,Ticker,Fund Name,1 MO,3 MO,YTD,1 YR,3 YR,5 YR,10 YR,As Of Date
0,SPY,SPDR S&P 500 ETF Trust,4.97%,11.99%,6.77%,29.29%,11.10%,14.57%,12.71%,02/22/2024
1,IVV,iShares Core S&P 500 ETF,5.01%,12.04%,6.79%,29.46%,11.16%,14.63%,12.77%,02/22/2024
2,VOO,Vanguard 500 Index Fund,5.01%,12.05%,6.82%,29.41%,11.16%,14.62%,12.77%,02/22/2024
3,VTI,Vanguard Total Stock Market ETF,4.68%,12.33%,6.16%,27.50%,8.90%,13.70%,12.08%,02/22/2024
4,QQQ,Invesco QQQ Trust,3.87%,12.59%,6.97%,49.90%,11.47%,21.27%,18.19%,02/22/2024
...,...,...,...,...,...,...,...,...,...,...
3395,BINV,Brandes International ETF,3.33%,8.47%,1.69%,--,--,--,--,02/22/2024
3396,BUSA,Brandes U.S. Value ETF,4.45%,11.97%,5.09%,--,--,--,--,02/22/2024
3397,BSMC,Brandes US Small-Mid Cap Value ETF,2.37%,9.32%,1.41%,--,--,--,--,02/22/2024
3398,MSTY,YieldMax MSTR Option Income Strategy ETF,--,--,--,--,--,--,--,NaN


In [10]:
# Alternando index para o Ticker do ETF

base_dados = base_dados.set_index("Ticker")
display(base_dados)

base_performance = base_performance.set_index("Ticker")
display(base_performance)

,Fund Name,Segment,1 YR Return,Expense Ratio,AUM
Ticker,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,29.29%,0.09%,$491.91B
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,29.46%,0.03%,$433.99B
VOO,Vanguard 500 Index Fund,Equity: U.S. - Large Cap,29.41%,0.03%,$404.83B
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,27.50%,0.03%,$366.51B
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,49.90%,0.20%,$244.72B
...,...,...,...,...,...
BINV,Brandes International ETF,Equity: Global - Total Market Value,--,0.70%,--
BUSA,Brandes U.S. Value ETF,Equity: U.S. - Total Market Value,--,0.60%,--
BSMC,Brandes US Small-Mid Cap Value ETF,Equity: U.S. - Extended Market Value,--,0.70%,--


,Fund Name,1 MO,3 MO,YTD,1 YR,3 YR,5 YR,10 YR,As Of Date
Ticker,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,4.97%,11.99%,6.77%,29.29%,11.10%,14.57%,12.71%,02/22/2024
IVV,iShares Core S&P 500 ETF,5.01%,12.04%,6.79%,29.46%,11.16%,14.63%,12.77%,02/22/2024
VOO,Vanguard 500 Index Fund,5.01%,12.05%,6.82%,29.41%,11.16%,14.62%,12.77%,02/22/2024
VTI,Vanguard Total Stock Market ETF,4.68%,12.33%,6.16%,27.50%,8.90%,13.70%,12.08%,02/22/2024
QQQ,Invesco QQQ Trust,3.87%,12.59%,6.97%,49.90%,11.47%,21.27%,18.19%,02/22/2024
...,...,...,...,...,...,...,...,...,...
BINV,Brandes International ETF,3.33%,8.47%,1.69%,--,--,--,--,02/22/2024
BUSA,Brandes U.S. Value ETF,4.45%,11.97%,5.09%,--,--,--,--,02/22/2024
BSMC,Brandes US Small-Mid Cap Value ETF,2.37%,9.32%,1.41%,--,--,--,--,02/22/2024


In [11]:
base_dados = base_dados[['Fund Name', 'Segment', 'Expense Ratio', 'AUM']]
base_performance = base_performance[['1 YR', '3 YR', '5 YR', '10 YR']]

display(base_dados)
display(base_performance)

,Fund Name,Segment,Expense Ratio,AUM
Ticker,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,0.09%,$491.91B
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,0.03%,$433.99B
VOO,Vanguard 500 Index Fund,Equity: U.S. - Large Cap,0.03%,$404.83B
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,0.03%,$366.51B
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,0.20%,$244.72B
...,...,...,...,...
BINV,Brandes International ETF,Equity: Global - Total Market Value,0.70%,--
BUSA,Brandes U.S. Value ETF,Equity: U.S. - Total Market Value,0.60%,--
BSMC,Brandes US Small-Mid Cap Value ETF,Equity: U.S. - Extended Market Value,0.70%,--


,1 YR,3 YR,5 YR,10 YR
Ticker,,,,
SPY,29.29%,11.10%,14.57%,12.71%
IVV,29.46%,11.16%,14.63%,12.77%
VOO,29.41%,11.16%,14.62%,12.77%
VTI,27.50%,8.90%,13.70%,12.08%
QQQ,49.90%,11.47%,21.27%,18.19%
...,...,...,...,...
BINV,--,--,--,--
BUSA,--,--,--,--
BSMC,--,--,--,--


### **Passo 4** - Fazer tratamento inicial dos dados, gerando um output final com uma base de dados a ser utilizada (ETAPA MAIS IMPORTANTE).

In [12]:
base_completa = base_dados.merge(base_performance, how='inner', on="Ticker")
display(base_completa)

,Fund Name,Segment,Expense Ratio,AUM,1 YR,3 YR,5 YR,10 YR
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,0.09%,$491.91B,29.29%,11.10%,14.57%,12.71%
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,0.03%,$433.99B,29.46%,11.16%,14.63%,12.77%
VOO,Vanguard 500 Index Fund,Equity: U.S. - Large Cap,0.03%,$404.83B,29.41%,11.16%,14.62%,12.77%
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,0.03%,$366.51B,27.50%,8.90%,13.70%,12.08%
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,0.20%,$244.72B,49.90%,11.47%,21.27%,18.19%
...,...,...,...,...,...,...,...,...
BINV,Brandes International ETF,Equity: Global - Total Market Value,0.70%,--,--,--,--,--
BUSA,Brandes U.S. Value ETF,Equity: U.S. - Total Market Value,0.60%,--,--,--,--,--
BSMC,Brandes US Small-Mid Cap Value ETF,Equity: U.S. - Extended Market Value,0.70%,--,--,--,--,--


In [ ]:
base_completa.to_excel("base_completa.xlsx")